In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [2]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [3]:
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000,)


In [5]:
x_train = x_train.reshape(-1,28*28).astype("float32")/255.0
x_test = x_test.reshape(-1,28*28).astype("float32")/255.0

In [7]:
# x_train = tf.convert_to_tensor(x_train)
# This is not required because it internally does this things


In [16]:
# Sequential API -> One Input maps to One Output

model = keras.Sequential(
    [
        keras.Input(shape=(28*28)),
        layers.Dense(512,activation="relu"),
        layers.Dense(256,activation = "relu"),
        layers.Dense(10),
    ]
)
print(model.summary())
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

# If we do not include the input layer in the sequential model, then we can not use the model.summary(). We can use it but after model.fit()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 512)               401920    
                                                                 
 dense_18 (Dense)            (None, 256)               131328    
                                                                 
 dense_19 (Dense)            (None, 10)                2570      
                                                                 
Total params: 535818 (2.04 MB)
Trainable params: 535818 (2.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [11]:
model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)

Epoch 1/5
1875/1875 - 16s - loss: 0.5747 - accuracy: 0.8278 - 16s/epoch - 8ms/step
Epoch 2/5
1875/1875 - 14s - loss: 0.2906 - accuracy: 0.9144 - 14s/epoch - 7ms/step
Epoch 3/5
1875/1875 - 14s - loss: 0.2228 - accuracy: 0.9340 - 14s/epoch - 7ms/step
Epoch 4/5
1875/1875 - 14s - loss: 0.1740 - accuracy: 0.9481 - 14s/epoch - 7ms/step
Epoch 5/5
1875/1875 - 14s - loss: 0.1411 - accuracy: 0.9576 - 14s/epoch - 7ms/step


In [12]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 1s - loss: 19.3229 - accuracy: 0.9372 - 921ms/epoch - 3ms/step


[19.322860717773438, 0.9372000098228455]

Another way to declare a model


In [33]:
model = keras.Sequential()
model.add(keras.Input(shape=(28*28)))
model.add(layers.Dense(512,activation="relu"))
print(model.summary())
model.add(layers.Dense(256,activation="relu",name="my_layer"))
print(model.summary())
model2.add(layers.Dense(10))
print(model.summary())

# Using model.summary() we can easily debug our model. We can check how the input shape changes over the layers

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_35 (Dense)            (None, 512)               401920    
                                                                 
Total params: 401920 (1.53 MB)
Trainable params: 401920 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_35 (Dense)            (None, 512)               401920    
                                                                 
 my_layer (Dense)            (None, 256)               131328    
                                                                 
Total params: 533248 (2.03 MB)
Trainable params: 533248 (2.03 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

## Difference between Functional API and Sequential API
## Limitation of Sequential API

### Functional API

In [25]:
# Functional API ( More Flexible => Support multiple input and multiple output)


inputs = keras.Input(shape=(28*28),name="Input_Layer")
x = layers.Dense(512,activation="relu",name="First_Dense_Layer")(inputs)
x = layers.Dense(256,activation="relu",name="Second_Dense_Layer")(x)
outputs = layers.Dense(10,activation="softmax",name="Output_Layer")(x)
model3 = keras.Model(inputs=inputs,outputs=outputs)

print(model3.summary())
model3.compile(
     loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_Layer (InputLayer)    [(None, 784)]             0         
                                                                 
 First_Dense_Layer (Dense)   (None, 512)               401920    
                                                                 
 Second_Dense_Layer (Dense)  (None, 256)               131328    
                                                                 
 Output_Layer (Dense)        (None, 10)                2570      
                                                                 
Total params: 535818 (2.04 MB)
Trainable params: 535818 (2.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [22]:
model3.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
model3.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/5
1875/1875 - 15s - loss: 0.5849 - accuracy: 0.8230 - 15s/epoch - 8ms/step
Epoch 2/5
1875/1875 - 15s - loss: 0.2945 - accuracy: 0.9133 - 15s/epoch - 8ms/step
Epoch 3/5
1875/1875 - 21s - loss: 0.2201 - accuracy: 0.9348 - 21s/epoch - 11ms/step
Epoch 4/5
1875/1875 - 20s - loss: 0.1712 - accuracy: 0.9489 - 20s/epoch - 11ms/step
Epoch 5/5
1875/1875 - 18s - loss: 0.1374 - accuracy: 0.9586 - 18s/epoch - 10ms/step
313/313 - 1s - loss: 17.3571 - accuracy: 0.9397 - 1s/epoch - 5ms/step


[17.35706901550293, 0.9397000074386597]

### Feature Extraction from layers

In [36]:
model = keras.Model(inputs=model.inputs,outputs=[model.layers[-2].output])
feature = model.predict(x_train)
print(feature.shape)

1875/1875 [==============================] - 5s 2ms/step
(60000, 512)


In [34]:
model = keras.Model(inputs=model.inputs,outputs=[model.get_layer('my_layer').output])
feature = model.predict(x_train)
print(feature.shape)

1875/1875 [==============================] - 6s 3ms/step
(60000, 256)
